In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import load_from_disk

from infodynamics.metrics import cos_sim, js_dist


DATA_PROCESSED = '../data/processed/'
DATA_INTERIM = '../data/interim/'

In [ ]:
# publications = load_from_disk(os.path.join(DATA_PROCESSED, 'publications'))
publications = load_from_disk(os.path.join(DATA_PROCESSED, 'tmp_abstracts'))
publications['doc_id'] = publications.map(lambda x: x['id'].split('_')[0])

In [ ]:
distances = []
for i in publications['doc_id']:
    abs = publications.filter(lambda x: x['id'] == str(i) + "_a")
    bod = publications.filter(lambda x: x['id'] == str(i) + "_b")
    dist = cos_sim(
        np.array(abs['embedding']), 
        np.array(bod['embedding'])
        )
    distances.append(dist)

In [ ]:
meta = pd.read_csv(os.path.join(DATA_INTERIM, 'publications_meta.csv'))
meta['distance'] = distances
meta['distance'].plot()